<a href="https://colab.research.google.com/github/Claudio-R/learning-tensorflow/blob/main/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart for beginners

## Set up TensorFlow

In [13]:
import tensorflow as tf
print('Tensorflow version:', tf.__version__)

Tensorflow version: 2.11.0


## Load a dataset

In [15]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## Build a machine learning model
Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the Flatten, Dense, and Dropout layers.

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [23]:
# We need ad extra dimension, so we use x_train[:1] instead of x_train[0] 
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.6742506 ,  0.01362085,  0.46963054,  0.6568645 , -0.4847511 ,
        -0.4082393 , -0.11609474, -0.23163164, -0.28940952, -0.22767837]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class.
It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is **discouraged** as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

In [37]:
tf.nn.softmax(predictions).numpy()

array([[0.05329464, 0.10602842, 0.167288  , 0.20173419, 0.06441428,
        0.06953618, 0.09312956, 0.08296798, 0.07831012, 0.08329662]],
      dtype=float32)

Define a loss function for training using losses.SparseCategoricalCrossentropy.
The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [39]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
loss_fn(y_train[:1], predictions).numpy()

2.665908

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [42]:
model.compile(optimizer='adam', loss=loss_fn, metrics='accuracy')

## Train and evaluate your model
Use the Model.fit method to adjust your model parameters and minimize the loss.

In [43]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2940 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1428 - accuracy: 0.9574
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1079 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0874 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0753 - accuracy: 0.9764


The Model.evaluate method checks the model's performance, usually on a validation set or test set.

In [44]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0750 - accuracy: 0.9784 - 638ms/epoch - 2ms/step


[0.07495329529047012, 0.9783999919891357]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [45]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
    ])

In [46]:
probability_model(x_test[:1])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[2.1692109e-08, 2.4142178e-08, 5.2801994e-07, 2.6236047e-04,
        2.6659074e-11, 1.3544098e-06, 1.9919004e-13, 9.9970752e-01,
        7.0044365e-08, 2.8042901e-05]], dtype=float32)>

# Quickstart for Experts

In [47]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.11.0


In [50]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


Use tf.data to batch and shuffle the dataset:

In [51]:
buffer_size = 100000
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

Build the tf.keras model using the Keras model subclassing API:

In [52]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(units=128, activation='relu')
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of MyModel
model = MyModel()

Choose an optimizer and loss function for training:

In [55]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [54]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use tf.GradientTape to train the model:

In [56]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

Test the model:

In [57]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [59]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_state()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.1355367749929428, Accuracy: 95.9566650390625, Test Loss: 0.05632271245121956, Test Accuracy: 98.1500015258789
Epoch 2, Loss: 0.04108615964651108, Accuracy: 98.7550048828125, Test Loss: 0.052743613719940186, Test Accuracy: 98.2699966430664
Epoch 3, Loss: 0.02146339602768421, Accuracy: 99.28500366210938, Test Loss: 0.05647468566894531, Test Accuracy: 98.23999786376953
Epoch 4, Loss: 0.013125917874276638, Accuracy: 99.57833099365234, Test Loss: 0.06669912487268448, Test Accuracy: 98.23999786376953
Epoch 5, Loss: 0.009185060858726501, Accuracy: 99.67666625976562, Test Loss: 0.06661085039377213, Test Accuracy: 98.27999877929688
